# Model Pendugaan Radiasi Matahari

## Pendahuluan 

Radiasi matahari sebagai sumber energi utama memiliki peran dalam pertukaran energi di atmosfer, lautan, dan daratan melalui siklus, yaitu hidrologi, karbon, dan energi. Berbagai fenomena cuaca bumi, seperti pembentukan awan hujan, siklon, dan turbulensi dipengaruhi oleh intensitas serta sebaran radiasi matahari yang diterima oleh bumi. Intensitas radiasi matahari yang diterima sampai atmosfer terluar bumi adalah $1366 \pm 7 ~ \text{W/m}^2$. Ini disebut juga sebagai konstanta penyinaran matahari dengan tetapan $1367.6 ~ \text{W/m}^2$ [@stull2015practical]. Sampai pada permukaan bumi, intensitas radiasi matahari (insolasi) menjadi berkurang menjadi seperempatnya, yaitu sekitar $341.4 \pm 1.2 ~ \text{W/m}^2$, dengan pembagian energi yang dipantulkan dan diserap secara berturut-turut sebesar $106.1 \pm 6.5 ~ \text{W/m}^2$ dan $233.8 \pm 3.8 ~ \text{W/m}^2$ [@trenberth2009earth]. 

Pengukuran radiasi matahari secara langsung masih terbilang sedikit dengan periode perekaman data historis yang pendek. Padahal, pengamatan radiasi matahari di permukaan bumi sangat penting dalam menjelaskan variabilitas iklim dan dampaknya pada perubahan iklim. Pada pemodelan iklim yang terus berkembang untuk menjelaskan kondisi iklim di masa lalu, saat ini dan masa depan tak lepas dari masukan data radiasi matahari dan konsentrasi karbon di atmosfer. Untuk mengetahui besaran nilai penduga radiasi matahari di suatu wilayah, model mekanistik maupun empirik dapat digunakan dengan masukan beberapa data historis variabel iklim lainnya seperti curah hujan dan suhu udara. 

Pada bagian pertama (@sec-modelinsolation), Anda akan mempelajari cara menghitung radiasi matahari ekstraterestrial (insolasi) menggunakan model mekanistik serta menampilkan sebaran nilainya berdasarkan letak lintang dan Julian Days. Bagian kedua (@sec-modelball) dan ketiga (@sec-modelhunt), Anda akan menghitung radiasi matahari yang diterima di permukaan bumi menggunakan model empirik. Data pendukung bagian kedua dan ketiga menggunakan data cuaca di Bandara Laguardia, New York, Amerika Serikat (`LaguardiaAirport-NYC.xlsx`). *Worksheet* ke-3 (`RawData`) adalah data yang akan diolah dengan berisikan 7 kolom: `DOY` (*Day of Year*, 1 sampai 365 atau 366), `YEAR` (tahun), `PRCP` (curah hujan, mm), `TAVG` (suhu udara rata-rata, °C), `TMAX` (suhu udara maksimum, °C), `TMIN` (suhu udara minimum, °C), dan `SRAD` (radiasi matahari langsung, W/m²). Data tersebut sudah dirapikan sehingga dapat langsung diolah. Periode data dibagi menjadi 2, yaitu untuk pembuatan dan validasi model dengan pemilihan tahun 1998-2018 dan 2019-2020, secara berturut-turut. Anda juga dapat mengganti pemilihan periode data sebagai latihan.

## Pendugaan Insolasi Radiasi Matahari Harian {#sec-modelinsolation}

Nilai insolasi matahari sebesar $341.4 ~ \text{W/m}^2$ merupakan rata-rata secara global. Terdapat variasi nilai tersebut berdasarkan letak geografis dan waktu. Untuk menduga nilai insolasi matahari di suatu wilayah, data yang diperlukan hanyalah letak lintang. Persamaan rata-rata harian insolasi matahari di suatu wilayah dapat dihitung dengan persamaan berikut.

$$
\bar{E} = \frac{S_0}{\pi} \cdot \left(\frac{a}{R} \right)^2 \cdot [h_0' \cdot sin(\phi) \cdot sin(\delta_s) + cos(\phi) \cdot cos(\delta_s) \cdot sin(h_0)]
$$

dengan simbol-simbol sebagai berikut:

- $\bar{E}$: rata-rata harian insolasi matahari ($\text{W/m}^2$)
- $S_0$: konstanta penyinaran matahari ($= 1366.7 ~ \text{W/m}^2$)
- $a$: rata-rata jarak bumi ke matahari ($= 149 ~ \text{Gm}$)
- $R$: jarak bumi ke matahari setiap hari (Gm)
- $h_0'$: sudut saat matahari terbit dan terbenam (radian)
- $\phi$: letak lintang 
- $\delta_s$: sudut deklinasi matahari 

Untuk mencari $R$, dapat digunakan persamaan berikut

$$
R = a \cdot \left(1 - e^2 \right) / \left(1 + e \cdot \text{cos}(v) \right)
$$

dengan e adalah eksentrisitas orbit bumi ($= 0.0167$) dan $v$ adalah sudut yang dibentuk oleh bumi, matahari, dan pusat orbit bumi. Nilai $v$ dapat dihitung dengan persamaan berikut

$$
v = 2 \pi \left(\frac{d - 4}{365.256363} \right)
$$

Sudut deklinasi matahari ($\delta_s$) dapat dihitung dengan persamaan berikut:

$$
\delta_s = 23.44° \cdot \text{cos} \left( \frac{360°(d - 172)}{365} \right)
$$

dengan $d$ adalah julian day. Sudut saat matahari terbit dan terbenam ($h_0$) dapat dihitung dengan persamaan berikut:

$$
h_0 = \text{arccos}(\text{MIN}[1, ~ \text{MAX}[-1, -\text{tan}(\phi) \cdot \text{tan}(\delta_s)]])
$$


#### Perhitungan {.unnumbered}

Untuk mengetahui sebaran nilai insolasi matahari, letak lintang yang digunakan dari -90° sampai 90° dengan jarak 1°. Julian day yang digunakan dari 1 sampai 365. 

#### R {.unnumbered}

1. Impor paket `ggplot2`, `metR`, dan `dplyr`. Jika belum terpasang, gunakan perintah 
   `install.packages("tidyverse")` dan `install.packages("metR")`.
   
   ```r
   library(ggplot2)
   library(dplyr)
   library(metR)
   ```

2. Membuat array letak lintang dan Julian day

   ```r
   jarak <- 1
   lat <- seq(-90, 90, jarak)
   doy <- seq(1, 365, jarak)
   ```

3. Membuat fungsi untuk menghitung insolasi harian matahari.

   ```r
    insolation <- function(lat, doy) {
        # Konstanta
        S0 <- 1367.6
        a <- 149

        # Konversi latitude ke radian
        lat_radian <- pi / 180 * lat

        # Menghitung jarak bumi ke matahari untuk setiap DOY
        v <- 2 * pi * (doy - 4) / 365.256363 # radian
        R <- a * (1 - 0.0167 ^ 2) / (1 + 0.0167 * cos(v)) # radian

        # Menghitung insolasi harian
        delta_s <- 23.44 * pi / 180 * cos(2 * pi * (doy - 172) / 365) 
        h0 <- acos(pmin(1, pmax(-1, -tan(lat_radian) * tan(delta_s))))
        E <- S0 / pi * 
            (a / R)^2 * 
            (
               h0 * sin(lat * pi / 180) * sin(delta_s) + 
               cos(lat * pi / 180) * cos(delta_s) * sin(h0)
            )

        return(E)
    }
    ```

4. Membuat *dataframe* untuk menyimpan hasil perhitungan

   ```r
   df <- as_tibble(expand.grid(lat = lat, doy = doy))
   df <- df %>% mutate(E_bar = insolation(lat, doy))
   ```

5. Membuat *plot* sebaran nilai insolasi matahari

   ```r
   kontur_50 <- seq(50, 550, 100)
   kontur_100 <- seq(100, 600, 100)

   ggplot(df, aes(x = doy, y = lat)) + 
    # Menambahkan garis kontur untuk per skala 100
    geom_contour(aes(z = E_bar, linetype = 'dashed'), show.legend = F, color = 'grey60', breaks = kontur_100) + 
    # Menambahkan garis kontur untuk per skala 50
    geom_contour(aes(z = E_bar, linetype = 'dotted'), show.legend = F, color = 'black', breaks = kontur_50) +
    # Menambahkan kontur terisi hanya untuk nilai E_bar = 0
    geom_contour_filled(aes(z = E_bar), breaks = c(0, 1), fill = 'black') +
    # Menambahkan teks pada kontur
    geom_text_contour(aes(z = E_bar), stroke = 0.3, skip = 0) +
    # Mengatur tema
    scale_x_continuous('Julian days', expand = c(0, 0), breaks = seq(0, 365, 30)) + 
    scale_y_continuous('Latitude (°)', expand = c(0, 0), breaks = seq(-90, 90, 10)) + 
    theme_bw()
   ```

![Variasi rata-rata harian insolasi matahari terhadap posisi lintang](Pictures/sun-insolation.png){#fig-insolation width=40%}



#### Python {.unnumbered}

--- (Akan dibuat) ---

1. Impor paket 
   
   ```python
   import numpy as np
   import pandas as pd
   
   import matplotlib.pyplot as plt
   import matplotlib.ticker as ticker
   ```

2. Membuat array letak lintang dan Julian day
   
   ```python
   jarak = 1
   lat = np.arange(-90, 90 + jarak, jarak)
   doy = np.arange(1, 365 + jarak, jarak)
   ```

3. Membuat fungsi insolasi matahari harian 
   
   ```python
   def insolation(lat, doy):
       # Konstanta
       S0 = 1367.6
       a = 149

       # Konversi latitude ke radian
       lat_radian = np.pi / 180 * lat

       # Menghitung jarak bumi ke matahari untuk setiap DOY
       v = 2 * np.pi * (doy - 4) / 365.256363 # radian
       R = a * (1 - 0.0167 ** 2) / (1 + 0.0167 * np.cos(v)) # radian

       # Menghitung insolasi harian
       delta_s = 23.44 * np.pi / 180 * np.cos(2 * np.pi * (doy - 172) / 365) 
       h0 = np.arccos(np.minimum(1, np.maximum(-1, -np.tan(lat_radian) * np.tan(delta_s))))
       E = S0 / np.pi * (a / R)**2 * (h0 * np.sin(lat * np.pi / 180) * np.sin(delta_s) + np.cos(lat * np.pi / 180) * np.cos(delta_s) * np.sin(h0))

       return E
   ```

4. Membuat tabel hasil perhitungan
   
   ```python
   df = pd.DataFrame({'lat': np.repeat(lat, len(doy)), 'doy': np.tile(doy, len(lat))})
   df['E_bar'] = insolation(df['lat'], df['doy'])
   ```

5. Membuat *plot* sebaran nilai insolasi matahari
   
   ```python
   fig, ax = plt.subplots(figsize=(6, 6))  # Set the figsize to (5, 5) for equal width and height
   cont = ax.tricontourf(df['doy'], df['lat'], df['E_bar'], levels=np.arange(0, 1, 0.1), cmap='Greys_r')
   cont_lines = ax.tricontour(df['doy'], df['lat'], df['E_bar'], levels=np.arange(0, 700, 100), colors='k')
   ax.tricontour(df['doy'], df['lat'], df['E_bar'], levels=np.arange(0, 700, 50), colors='k', linestyles='dotted')
   ax.set_xlabel('Julian days')
   ax.set_ylabel('Latitude (°)')
   ax.set_xlim(0, 365)
   ax.set_ylim(-90, 90)
   ax.xaxis.set_major_locator(ticker.MultipleLocator(30))
   ax.yaxis.set_major_locator(ticker.MultipleLocator(10))

   # Add labels to contour lines with background color
   clabels = ax.clabel(cont_lines, inline=True, fontsize=6)
   for label in clabels:
       label.set_bbox({'facecolor': 'white', 'edgecolor': 'white'})
   ```

6. Untuk menyimpan gambar, dapat menggunakan perintah berikut

   ```python
   plt.savefig('sun-insolation-py.png', dpi=300) 
   plt.show()
   ```

![Variasi rata-rata harian insolasi matahari terhadap posisi lintang](Pictures/sun-insolation-py.png){#fig-insolation-py width=40%}

## Model Pendugaan Ball et al. (2004) {#sec-modelball}

Ball et al. [@ball2004] membangun model empiris untuk menduga radiasi matahari di permukaan bumi dengan masukan suhu udara maksimum, suhu udara minimum, curah hujan, dan *julian days*. Persamaan yang digunakan berupa regresi linier berganda, yaitu

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + ... + \beta_{12} X_{12}
$$

di mana keterangan setiap variabel bebas dapat dilihat pada tabel berikut.

| Prediktor |                Keterangan                |
| :-------: | :--------------------------------------: |
|  $X_{1}$  |             Curah hujan (mm)             |
|  $X_{2}$  |         Suhu udara maksimum (°C)         |
|  $X_{3}$  |         Suhu udara minimum (°C)          |
|  $X_{4}$  |               Day of Year                |
|  $X_{5}$  |              (Curah hujan)²              |
|  $X_{6}$  |          (Suhu udara maksimum)²          |
|  $X_{7}$  |          (Suhu udara minimum)²           |
|  $X_{8}$  |              (Day of Year)²              |
|  $X_{9}$  |     Curah hujan * Suhu udara minimum     |
| $X_{10}$  | Suhu udara maksimum * Suhu udara minimum |
| $X_{11}$  |    Curah hujan * Suhu udara maksimum     |
| $X_{12}$  |    Suhu udara maksimum * Day of Year     |

: Keterangan variabel bebas dari persamaan empirik [@ball2004] {#tbl-modelball}

#### Pengolahan Data {.unnumbered}

#### R {.unnumbered}

1. Package yang digunakan dalam pengolahan data di R adalah `tidyverse` dan `readxl`. Jika Anda belum memasang package ini, gunakan perintah berikut. 
   ```r
   install.packages(c("tidyverse", "readxl"))
   ```
   
2. Impor data excel (`LaguardiaAirport-NYC.xlsx`) pada sheet `RawData` dengan perintah berikut. 
   
   ```r
   dpath <- "data/" # Lokasi folder data
   data <- read_excel(paste0(dpath, "LaguardiaAirport-NYC.xlsx"), sheet = "RawData")
   data
   ```

   ```
   # A tibble: 8,401 × 7
       YEAR   DOY  PRCP  TAVG  TMAX  TMIN  SRAD
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
    1  1998     1   0    -5    -1.1  -8.9 107.
    2  1998     2   0     4.7  10    -0.6  82.8
    3  1998     3   0    12    15.6   8.3  74.7
    4  1998     4   0    11.7  17.2   6.1  78.8
    5  1998     5   0     5.9   6.7   5    75.6
    6  1998     6   1    10.3  15     5.6  34.3
    7  1998     7  28.7   9.5  13.9   5    26.8
    8  1998     8   0.8  10.3  16.1   4.4  37.5
    9  1998     9   0    12    15     8.9  46.8
   10  1998    10   0     7.8  10     5.6  97.6
   # … with 8,391 more rows
   # ℹ Use `print(n = ...)` to see more rows
   ```
  
3. Pilih periode tahun untuk pembuatan dan validasi model dengan perintah berikut. 
   
   ```r
   # Pembuatan model
   data_train <- data %>% filter(YEAR <= 2018)
   
   # Validasi model
   data_test <- data %>% filter(YEAR >= 2019)
   ```

4. Lakukan perhitungan prediktor ke-5 sampai ke-12 sesuai dengan [@tbl-modelball]
   
   ```r
   data_train <- data_train %>%
      mutate(
         PRCP_sq = PRCP^2, TMAX_sq = TMAX^2, TMIN_sq = TMIN^2, DOY_sq  = DOY^2,
         PRCP_TMAX = PRCP * TMAX, PRCP_TMIN = PRCP * TMIN,
         TMAX_TMIN = TMAX * TMIN, TMAX_DOY  = TMAX * DOY
      )
   
   data_test <- data_test %>%
      mutate(
         PRCP_sq = PRCP^2, TMAX_sq = TMAX^2, TMIN_sq = TMIN^2, DOY_sq  = DOY^2,
         PRCP_TMAX = PRCP * TMAX, PRCP_TMIN = PRCP * TMIN,
         TMAX_TMIN = TMAX * TMIN, TMAX_DOY  = TMAX * DOY
      )
   ```
   
5. Lakukan pembuatan model regresi linier berganda pada data `data_train` sesuai dengan model Ball ([@tbl-modelball]). 
   
   ```r
   model <- lm(SRAD ~ PRCP + TMAX + TMIN + DOY + 
                      PRCP_sq + TMAX_sq + TMIN_sq + DOY_sq + 
                      PRCP_TMAX + PRCP_TMIN + TMAX_TMIN + TMAX_DOY, 
               data = data_train)
   summary(model)
   ```

   ```
   Call:
   lm(formula = SRAD ~ PRCP + TMAX + TMIN + DOY + PRCP_sq + TMAX_sq +
       TMIN_sq + DOY_sq + PRCP_TMAX + PRCP_TMIN + TMAX_TMIN + TMAX_DOY,
       data = data_train)

   Residuals:
        Min       1Q   Median       3Q      Max
   -230.598  -30.858    2.829   32.497  189.113

   Coefficients:
                 Estimate Std. Error t value Pr(>|t|)
   (Intercept) -4.120e+01  4.068e+00 -10.126  < 2e-16 ***
   PRCP        -3.315e+00  1.933e-01 -17.151  < 2e-16 ***
   TMAX         2.259e+01  8.621e-01  26.203  < 2e-16 ***
   TMIN        -2.705e+01  8.212e-01 -32.945  < 2e-16 ***
   DOY          1.988e+00  4.015e-02  49.508  < 2e-16 ***
   PRCP_sq      2.469e-02  1.324e-03  18.649  < 2e-16 ***
   TMAX_sq     -9.876e-01  4.577e-02 -21.577  < 2e-16 ***
   TMIN_sq     -1.481e+00  5.217e-02 -28.386  < 2e-16 ***
   DOY_sq      -5.281e-03  1.029e-04 -51.317  < 2e-16 ***
   PRCP_TMAX   -1.217e-01  1.966e-02  -6.189 6.35e-10 ***
   PRCP_TMIN    1.448e-01  2.069e-02   6.998 2.82e-12 ***
   TMAX_TMIN    2.602e+00  9.450e-02  27.534  < 2e-16 ***
   TMAX_DOY    -1.184e-02  7.625e-04 -15.534  < 2e-16 ***
   ---
   Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

   Residual standard error: 47.29 on 7657 degrees of freedom
   Multiple R-squared:  0.7423,    Adjusted R-squared:  0.7419
   F-statistic:  1838 on 12 and 7657 DF,  p-value: < 2.2e-16
   ```

   Bisa Anda lihat pada bagian `Multiple R-squared` untuk mengetahui nilai dari koefisien determinasi (R²). Semua prediktor yang digunakan untuk mengestimasi radiasi matahari signifikan (p-value $< 0.05$) dengan nilai R²=74%. 

6. Anda dapat melakukan estimasi nilai radiasi matahari pada model yang sudah dibangun dengan menggunakan data validasi (`data_test`) dengan perintah berikut. 
   
   ```r
   data_test <- data_test %>%
      mutate(SRAD_pred = predict(model, data_test))
   ```

7. Kemudian, lakukan perhitungan koefisien determinasi (R²) dan korelasi Pearson (r) pada `data_test` dengan perintah berikut. 
   
   ```r
   # korelasi Pearson (r)
   r <- cor(data_test$SRAD, data_test$SRAD_pred)

   # koefisien determinasi (R²)
   R2 <- r^2

   # Print
   print(r); print(R2)
   ```

   ```
   [1] 0.8508384
   [1] 0.7239259
   ```

   Atau bisa juga dengan melakukan plot antara nilai aktual dan prediksi radiasi matahari dengan menampilkan garis regresi beserta persamaan regresinya ditambah dengan nilai R² dan r ([@fig-sunplotball1]).

   ```r
   ggplot(data_test, aes(x = SRAD_pred, y = SRAD)) +
      geom_point() +
      geom_abline(intercept = 0, slope = 1, color = "blue") +
      labs(y = "Observasi (W/m²)", x = "Model (W/m²)")
   ```

![Perbandingan radiasi matahari aktual dengan model (Ball)](Pictures/sun-radiation-actualvspredicted.png){#fig-sunplotball1}

8. Jika Anda perhatikan pada [@fig-sunplotball1], nilai radiasi matahari tidak mungkin bernilai negatif dan ini umum terjadi saat menggunakan model regresi. Oleh karena itu, nilai radiasi matahari yang bernilai negatif diubah menjadi 0 ([@fig-sunplotball2]). 
   
   ```r
   data_test <- data_test %>%
      mutate(SRAD_pred = ifelse(SRAD_pred < 0, 0, SRAD_pred))

   ggplot(data_test, aes(x = SRAD_pred, y = SRAD)) +
      geom_point() +
      geom_abline(intercept = 0, slope = 1, color = "blue") +
      scale_x_continuous(limits = c(0, 400), expand = c(0, 0)) +
      scale_y_continuous(limits = c(0, 400), expand = c(0, 0)) +
      labs(y = "Observasi (W/m²)", x = "Model (W/m²)") 
   ```

![Perbandingan radiasi matahari aktual dengan model (Ball)](Pictures/sun-radiation-actualvspredicted-corr.png){#fig-sunplotball2}


#### Python {.unnumbered}

1. Impor package `pandas`, `sklearn`, dan `matplotlib`. Anda perlu memasang `openpyxl` jika belum memasangnya. 
   
   ```bash
   pip install openpyxl
   ```

   ```python
   import pandas as pd
   import matplotlib.pyplot as plt
   from sklearn.linear_model import LinearRegression
   from sklearn import metrics
   ```
   
2. Impor data dengan perintah berikut.
   
   ```python
   path = 'data'
   data = pd.read_excel(f'{path}/LaguardiaAirport-NYC.xlsx', sheet_name='RawData')
   data
   ```

   ```
         YEAR  DOY  PRCP  TAVG  TMAX  TMIN   SRAD
   0     1998    1   0.0  -5.0  -1.1  -8.9  106.8
   1     1998    2   0.0   4.7  10.0  -0.6   82.8
   2     1998    3   0.0  12.0  15.6   8.3   74.7
   3     1998    4   0.0  11.7  17.2   6.1   78.8
   4     1998    5   0.0   5.9   6.7   5.0   75.6
   ...    ...  ...   ...   ...   ...   ...    ...
   8396  2020  362   0.0  -0.2   4.4  -2.7   99.4
   8397  2020  363   0.0   6.1  11.1   2.2   61.3
   8398  2020  364   0.0   5.6   7.2   0.0   97.1
   8399  2020  365   0.0   1.8   7.2  -1.0   56.5
   8400  2020  366  13.5   7.7  10.0   3.3   24.0
   
   [8401 rows x 7 columns]
   ```

3. Pilih periode tahun untuk pembuatan dan validasi model dengan perintah berikut. 
   
   ```python
   # Pembuatan model
   data_train = data[data['YEAR'] < 2019]
   # Validasi model
   data_test = data[data['YEAR'] >= 2019]
   ```

4. Lakukan perhitungan prediktor ke-5 sampai ke-12 sesuai dengan [@tbl-modelball]
   
   ```python
   data_train.loc[:, 'PRCP_sq']   = data_train.loc[:, 'PRCP'] ** 2
   data_train.loc[:, 'TMAX_sq']   = data_train.loc[:, 'TMAX'] ** 2
   data_train.loc[:, 'TMIN_sq']   = data_train.loc[:, 'TMIN'] ** 2
   data_train.loc[:, 'DOY_sq']    = data_train.loc[:, 'DOY']  ** 2
   data_train.loc[:, 'PRCP_TMAX'] = data_train.loc[:, 'PRCP'] * data_train.loc[:, 'TMAX']
   data_train.loc[:, 'PRCP_TMIN'] = data_train.loc[:, 'PRCP'] * data_train.loc[:, 'TMIN']
   data_train.loc[:, 'TMAX_TMIN'] = data_train.loc[:, 'TMAX'] * data_train.loc[:, 'TMIN']
   data_train.loc[:, 'TMAX_DOY']  = data_train.loc[:, 'TMAX'] * data_train.loc[:, 'DOY']
   ```
   
5. Lakukan pembuatan model regresi linier berganda pada data `data_train` sesuai dengan model Ball ([@tbl-modelball]). 
   
   ```python
   # Model regresi linier
   model = LinearRegression()

   model.fit(
      # Prediktor
      data_train.loc[:, ['PRCP', 'TAVG', 'TMAX', 'TMIN', 
                         'PRCP_sq', 'TMAX_sq', 'TMIN_sq', 
                         'DOY_sq', 'PRCP_TMAX', 'PRCP_TMIN', 
                         'TMAX_TMIN', 'TMAX_DOY']],
      # Prediktan
      data_train.loc[:, 'SRAD']
   )
   ```

6. Oleh karena respon dari package `sklearn` tidak bisa menghasilkan *summary* seperti pada R, Anda perlu membuat fungsi tersendiri. Pada fungsi `metrics`, telah tersedia beberapa metrik yang dapat digunakan untuk mengevaluasi model. 
   
   ```python
   def summary(model, x, y):
      # Estimasi
      y_pred = model.predict(x)
      
      # Jika terdapat nilai negatif, maka nilai tersebut akan diubah menjadi 0
      y_pred[y_pred < 0] = 0

      # Metrik
      r2   = metrics.r2_score(y, y_pred)
      rmse = metrics.mean_squared_error(y, y_pred, squared=False)
      mae  = metrics.mean_absolute_error(y, y_pred)
      
      print(f'R²   = {round(r2, 2)}')
      print(f'RMSE = {round(rmse, 2)}')
      print(f'MAE  = {round(mae, 2)}')
   
   summary(model, 
      data_train.loc[:, ['PRCP', 'TAVG', 'TMAX', 'TMIN', 
                         'PRCP_sq', 'TMAX_sq', 'TMIN_sq', 
                         'DOY_sq', 'PRCP_TMAX', 'PRCP_TMIN', 
                         'TMAX_TMIN', 'TMAX_DOY']],
      data_train.loc[:, 'SRAD']
   )
   ```
   ```
   R²   = 0.74
   RMSE = 47.25
   MAE  = 37.6
   ```

7. Untuk memvalidasi model, Anda perlu menghitung prediktor ke-5 sampai ke-12 pada data `data_test` sesuai dengan [@tbl-modelball]. Kemudian, gunakan `model.predict()` untuk menghitung prediksi pada data `data_test`.
   
   ```python
   data_test.loc[:, 'PRCP_sq']   = data_test.loc[:, 'PRCP'] ** 2
   data_test.loc[:, 'TMAX_sq']   = data_test.loc[:, 'TMAX'] ** 2
   data_test.loc[:, 'TMIN_sq']   = data_test.loc[:, 'TMIN'] ** 2
   data_test.loc[:, 'DOY_sq']    = data_test.loc[:, 'DOY']  ** 2
   data_test.loc[:, 'PRCP_TMAX'] = data_test.loc[:, 'PRCP'] * data_test.loc[:, 'TMAX']
   data_test.loc[:, 'PRCP_TMIN'] = data_test.loc[:, 'PRCP'] * data_test.loc[:, 'TMIN']
   data_test.loc[:, 'TMAX_TMIN'] = data_test.loc[:, 'TMAX'] * data_test.loc[:, 'TMIN']
   data_test.loc[:, 'TMAX_DOY']  = data_test.loc[:, 'TMAX'] * data_test.loc[:, 'DOY']

   summary(model, 
      data_test.loc[:, ['PRCP', 'TAVG', 'TMAX', 'TMIN', 
                        'PRCP_sq', 'TMAX_sq', 'TMIN_sq', 
                        'DOY_sq', 'PRCP_TMAX', 'PRCP_TMIN', 
                        'TMAX_TMIN', 'TMAX_DOY']],
      data_test.loc[:, 'SRAD']
   )
   ```

   ```
   R²   = 0.72
   RMSE = 51.69
   MAE  = 40.72
   ```

8. Untuk membuat grafik dari `data_test` maupun `data_train`, Anda dapat menggunakan `matplotlib` ([@fig-sunmodelballpy])
   
   ```python
   y_pred = model.predict(data_train.loc[:, ['PRCP', 'TMAX', 'TMIN', 'DOY', 
                                             'PRCP_sq', 'TMAX_sq', 'TMIN_sq', 
                                             'DOY_sq', 'PRCP_TMAX', 'PRCP_TMIN', 
                                             'TMAX_TMIN', 'TMAX_DOY']])

   # Grafik data train
   plt.figure(figsize=(10, 10))
   plt.scatter(data_train.loc[:, 'SRAD'], y_pred)

   plt.xlim(0, 380)
   plt.ylim(0, 380)

   # Menambahkan label pada sumbu x dan y
   plt.xlabel('Model (W/m²)')
   plt.ylabel('Observasi (W/m²)')

   plt.show()
   ```

![Perbandingan radiasi matahari aktual terhadap model (Ball)](Pictures/sun-radiation-actualvspredicted-py.png){#fig-sunmodelballpy}

## Model Pendugaan Hunt et al. (1998) {#sec-modelhunt}

Model pendugaan radiasi matahari lain adalah oleh Hunt [@hunt1998], yaitu gabungan model mekanistik dan empirik dengan masukan suhu udara maksimum, suhu udara minimum, dan curah hujan. Model mekanistik digunakan untuk menduga radiasi matahari yang berada di permukaan atmosfer (radiasi ekstraterestrial, $S_0$). Model pendugaan Hunt (1998) adalah sebagai berikut.

$$
R_s = a_0 S_0 (T_{max} - T_{min})^{0.5} + a_1 T_{max} + a_2 P + a_3 P^2 + a_4
$$

dimana $R_s$ adalah radiasi matahari harian ($MJ~m^{−2}~hari^{−1}$), $S_0$ adalah radiasi matahari di puncak atmosfer ($MJ~m^{−2}~hari^{−1}$), $P$ adalah curah hujan (mm), $T_{max}$ adalah suhu udara maksimum ($^o C$), dan $T_{min}$ adalah suhu udara minimum ($^o C$). Untuk mengestimasi nilai $S_0$, Hunt menggunakan persamaan mekanistik dalam Spitters [@spitters1986], yaitu:

$$
S_0 = S_{sc} \left[ 1 + 0.033 \cos \left( \frac{360~t_d}{365} \right) \right] sin(β)
$$

dimana, $S_0$ adalah irradiasi ekstra terestrial ($\text{W/m}^2$), $S_{sc}$ konstanta matahari ($1370 ~ \text{W/m}^2$), suku $cos$ adalah jarak tahunan antara bumi dan matahari yang dinyatakan dalam derajat, $t_d$ adalah julian day, dan $sin(β)$ adalah sinus sudut elevasi matahari (satuan detik) yang didefinisikan pada persamaan:

$$
sin(β) = 3600 \left[ D ~ sin(λ) ~ sin(δ) + \frac{24}{\pi} ~ cos(λ) ~ cos(δ) ~ \sqrt{(1 - tan^2(λ) ~ tan^2(δ))} \right]
$$

dimana, $λ$ adalah letak lintang dari lokasi stasiun dan δ adalah sudut deklinasi matahari pada saat julian day dan dinyatakan dalam derajat dengan estimasi pada persamaan:

$$
sin(δ) = -sin(23.45) ~ cos \left( \frac{360~(t_d + 10)}{365} \right)
$$

dan $D$ adalah panjang hari (jam) dengan persamaan:

$$
D = 12 + \frac{24}{180} arcsin(tan(λ) ~ tan(δ))
$$

#### Pengolahan Data {.unnumbered}

#### R {.unnumbered}

Langkah-langkah pembuatan model radiasi matahari dengan menggunakan model Hunt et al. (1998) adalah sebagai berikut.

1. Impor data Excel. Caranya sama seperti pada subbab sebelumnya.
   
   ```r
   dpath <- "data/" # Lokasi folder data
   data <- read_excel(paste0(dpath, "LaguardiaAirport-NYC.xlsx"), sheet = "RawData")
   data
   ```

   ```
   # A tibble: 8,401 × 7
       YEAR   DOY  PRCP  TAVG  TMAX  TMIN  SRAD
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
    1  1998     1   0    -5    -1.1  -8.9 107.
    2  1998     2   0     4.7  10    -0.6  82.8
    3  1998     3   0    12    15.6   8.3  74.7
    4  1998     4   0    11.7  17.2   6.1  78.8
    5  1998     5   0     5.9   6.7   5    75.6
    6  1998     6   1    10.3  15     5.6  34.3
    7  1998     7  28.7   9.5  13.9   5    26.8
    8  1998     8   0.8  10.3  16.1   4.4  37.5
    9  1998     9   0    12    15     8.9  46.8
   10  1998    10   0     7.8  10     5.6  97.6
   # … with 8,391 more rows
   # ℹ Use `print(n = ...)` to see more rows
   ```

2. Sebelum Anda melakukan pembuatan model regresi linier berganda, Anda harus melakukan perhitungan $S_0$ terlebih dahulu. Kami menyediakan fungsi untuk menghitung nilai $S_0$ seperti pada [@sec-modelhunt]. Anda hanya memasukkan nilai latitude serta *day of year* pada fungsi ini. 
   
   ```r
   S0 <- function(lat, doy){
      # Fungsi untuk menghitung sin(delta)
      sin_delta <- -sinpi(23.45 / 180) * cospi((360 * (doy + 10) / 365) / 180)
      asin_delt <- asin(sin_delta) * 180 / pi
      
      # sin(lat) * sin(delta)
      s_lat_delt <- sinpi(lat / 180) * sinpi(asin_delt / 180)
      
      # cos(lat) * cos(delta)
      c_lat_delt <- cospi(lat / 180) * cospi(asin_delt / 180)
      
      # (sin(lat) * sin(delta)) / (cos(lat) * cos(delta))
      t_lat_delt <- s_lat_delt / c_lat_delt
      
      # Fungi perhitungan panjang hari (D)
      D <- 12 + 24/180 * asin(t_lat_delt) * 180 / pi
      
      # Fungsi perhitungan sudut elevasi matahari
      sin_beta <- 3600 * (D * s_lat_delt + 24/pi * c_lat_delt * sqrt(1 - t_lat_delt^2))
      
      # Fungsi radiasi matahari ekstra terestrial
      S_0 <- 1370 * (1 + 0.033 * cospi(360 / 180 * doy / 365)) * sin_beta
      
      # Konversi J/m² ke MJ/m²
      return(S_0 / 1000000)
   }
   ```

3. Kemudian, tambahkan kolom baru pada `data` yang berisi nilai $S_0$ dengan perintah berikut. 
   
   ```r
   data <- data %>%
      mutate(
         S0 = S0(40.77945, DOY) * 11.57407 # konversi dari MJ/m² ke W/m²
      )
   ```

4. Untuk membuat model, lakukan pembagian data menjadi `data_test` dan `data_train` dengan perintah berikut. 
   
   ```r
   data_train <- data %>% filter(YEAR <= 2018)
   data_test <- data %>% filter(YEAR >= 2019)
   ```

5. Hitunglah `S0 * (Tmax - Tmin)^0.5` dan `PRCP^2` dengan menambahkan dua kolom baru pada `data_train` dan `data_test` dengan perintah berikut. 
   
   ```r
   data_train <- data_train %>%
      mutate(
         TMAX_TMIN = S0 * (TMAX - TMIN)^0.5,
         PRCP_sq = PRCP^2
      )
   
   data_test <- data_test %>%
      mutate(
         TMAX_TMIN = S0 * (TMAX - TMIN)^0.5,
         PRCP_sq = PRCP^2
      )
   ```
   
6. Buat model regresi linier berganda 
   
   ```r
   model <- lm(SRAD ~ TMAX_TMIN + TMAX + PRCP + PRCP_sq, data = data_train)
   summary(model)
   ```

7. Kemudian, lakukan prediksi pada `data_test` dan evaluasi hasil estimasi model menggunakan metrik $R^2$ dan korelasi Pearson dengan perintah berikut .
   
   ```r
   data_test <- data_test %>%
      mutate(SRAD_pred = predict(model, data_test))

   # korelasi Pearson (r)
   r <- cor(data_test$SRAD, data_test$SRAD_pred)

   # koefisien determinasi (R²)
   R2 <- r^2

   print(r); print(R2)
   ```

   ```
   [1] 0.8474758
   [1] 0.7182153
   ```

8. Sama seperti Ball, model Hunt juga dapat dipastikan estimasi radiasi matahari bernilai negatif. Untuk mengubahnya menjadi 0, lakukan hal yang sama seperti pada model Ball pada langkah ke-7 dan buat grafiknya ([@fig-sunmodelhunt])

   ```r
   data_test <- data_test %>%
      mutate(SRAD_pred = ifelse(SRAD_pred < 0, 0, SRAD_pred))

   ggplot(data_test, aes(x = SRAD_pred, y = SRAD)) +
      geom_point() +
      geom_abline(intercept = 0, slope = 1, color = "blue") +
      scale_x_continuous(limits = c(0, 400), expand = c(0, 0)) +
      scale_y_continuous(limits = c(0, 400), expand = c(0, 0)) +
      labs(y = "Observasi (W/m²)", x = "Model (W/m²)") 
   ```

![Perbandingan radiasi matahari aktual terhadap model (Hunt)](Pictures/sun-radiation-actualvspredicted-hunt.png){#fig-sunmodelhunt}


#### Python {.unnumbered}

1. Impor package 
   
   ```python
   import pandas as pd
   import matplotlib.pyplot as plt
   from sklearn.linear_model import LinearRegression
   from sklearn import metrics
   ```

2. Impor data Excel. Caranya sama seperti pada subbab sebelumnya.
   
   ```python
   path = 'data'
   data = pd.read_excel(f'{path}/LaguardiaAirport-NYC.xlsx', sheet_name='RawData')
   data
   ```

   ```
         YEAR  DOY  PRCP  TAVG  TMAX  TMIN   SRAD
   0     1998    1   0.0  -5.0  -1.1  -8.9  106.8
   1     1998    2   0.0   4.7  10.0  -0.6   82.8
   2     1998    3   0.0  12.0  15.6   8.3   74.7
   3     1998    4   0.0  11.7  17.2   6.1   78.8
   4     1998    5   0.0   5.9   6.7   5.0   75.6
   ...    ...  ...   ...   ...   ...   ...    ...
   8396  2020  362   0.0  -0.2   4.4  -2.7   99.4
   8397  2020  363   0.0   6.1  11.1   2.2   61.3
   8398  2020  364   0.0   5.6   7.2   0.0   97.1
   8399  2020  365   0.0   1.8   7.2  -1.0   56.5
   8400  2020  366  13.5   7.7  10.0   3.3   24.0
   
   [8401 rows x 7 columns]
   ```

3. Sebelum Anda melakukan pembuatan model regresi linier berganda, Anda harus melakukan perhitungan $S_0$ terlebih dahulu. Kami menyediakan fungsi untuk menghitung nilai $S_0$ seperti pada [@sec-modelhunt]. Anda hanya memasukkan nilai latitude serta *day of year* pada fungsi ini. Perlu modul tambahan `numpy` untuk menghitung nilai trigonometri. 
   
   ```python
   from numpy import sin, cos, pi, arcsin, sqrt

   def S0(lat, doy):
      # Fungsi untuk menghitung sin(delta)
      sin_delta = -sin(pi * 23.45 / 180) * cos(pi * (360 * (doy + 10) / 365) / 180)
      asin_delt = arcsin(sin_delta) * 180 / pi
      
      # sin(lat) * sin(delta)
      s_lat_delt = sin(pi * lat / 180) * sin(pi * asin_delt / 180)
      
      # cos(lat) * cos(delta)
      c_lat_delt = cos(pi * lat / 180) * cos(pi * asin_delt / 180)
      
      # (sin(lat) * sin(delta)) / (cos(lat) * cos(delta))
      t_lat_delt = s_lat_delt / c_lat_delt
      
      # Fungi perhitungan panjang hari (D)
      D = 12 + 24/180 * arcsin(t_lat_delt) * 180 / pi
      
      # Fungsi perhitungan sudut elevasi matahari
      sin_beta = 3600 * (D * s_lat_delt + 24/pi * c_lat_delt * sqrt(1 - t_lat_delt**2))
      
      # Fungsi perhitungan radiasi matahari
      S_0 = 1370 * (1 + 0.033 * cos(pi * 360 / 180 * doy / 365)) * sin_beta
      
      # Konversi J/m² ke MJ/m²
      return S_0 / 1000000
   ```

4. Kemudian, tambahkan kolom baru pada `data` yang berisi nilai $S_0$ dengan perintah berikut. 
   
   ```python
   data.loc[:, 'S0'] = S0(40.77945, data.loc[:, 'DOY']) * 11.57407
   ```

5. Untuk membuat model, lakukan pembagian data menjadi `data_test` dan `data_train` dengan perintah berikut. 
   
   ```python
   # Pembuatan model
   data_train = data[data['YEAR'] < 2019]
   # Validasi model
   data_test = data[data['YEAR'] >= 2019]
   ```

6. Hitunglah `S0 * (Tmax - Tmin)^0.5` dan `PRCP^2` dengan menambahkan dua kolom baru pada `data_train` dan `data_test` dengan perintah berikut. 
   
   ```python
   # Data train
   data_train.loc[:, 'S0_TMAX_TMIN'] = data_train.loc[:, 'S0'] * sqrt(data_train.loc[:, 'TMAX'] - data_train.loc[:, 'TMIN'])
   data_train.loc[:, 'PRCP_sq'] = data_train.loc[:, 'PRCP'] ** 2

   # Data test
   data_test.loc[:, 'S0_TMAX_TMIN'] = data_test.loc[:, 'S0'] * sqrt(data_test.loc[:, 'TMAX'] - data_test.loc[:, 'TMIN'])
   data_test.loc[:, 'PRCP_sq'] = data_test.loc[:, 'PRCP'] ** 2
   ```
   
7. Buat model regresi linier berganda 
   
   ```python
   model = LinearRegression()
   model.fit(
      # Prediktor   
      data_train.loc[:, ['S0_TMAX_TMIN', 'PRCP', 'PRCP_sq']], 
      # Prediktan
      data_train.loc[:, 'SRAD']
   )
   ```

8. Kemudian, lakukan validasi model dengan menggunakan `data_test`. 
   
   ```python
   summary(model, 
      data_test.loc[:, ['S0_TMAX_TMIN', 'PRCP', 'PRCP_sq']], 
      data_test.loc[:, 'SRAD']
   )
   ```

   ```
   R²   = 0.71
   RMSE = 52.53
   MAE  = 41.64
   ```

9.  Terakhir, lakukan pembuatan grafik dari `data_test` ([@fig-sunmodelhuntpy])
   
   ```python
   y_pred = model.predict(data_test.loc[:, ['S0_TMAX_TMIN', 'PRCP', 'PRCP_sq']])
   y_pred[y_pred < 0] = 0

   # Visualize
   fig = plt.figure(figsize=(4, 4))

   # Make plot scatter with small size
   plt.scatter(y_pred, data_test.loc[:, 'SRAD'], s=1)

   # Make line plot 1:1
   plt.plot([0, 400], [0, 400], color='red')

   # expand plot area
   plt.xlim(0, 380)
   plt.ylim(0, 380)

   # Set axis labels
   plt.xlabel('Model (W/m²)')
   plt.ylabel('Observasi (W/m²)')
   plt.show()
   ```

![Perbandingan radiasi matahari aktual terhadap model (Hunt)](Pictures/sun-radiation-actualvspredicted-Hunt-py.png){#fig-sunmodelhuntpy}